In [1]:
# %matplotlib inline

In [42]:
import numpy as np
import trimesh
import os

from bfieldtools.mesh_conductor import MeshConductor
from bfieldtools.contour import scalar_contour
from bfieldtools.utils import combine_meshes, find_mesh_boundaries

In [3]:
resolution = ''
resolution = '_coarse'
coilmeshes = [
    'coil-x'+resolution+'.stl',
    'coil-y'+resolution+'.stl',
    'coil+x'+resolution+'.stl',
    'coil+y'+resolution+'.stl'
]

contour_file = 'Imedco_n20_G10_coarse.txt'

In [4]:
planes = []
for stl in coilmeshes:
    planes.append(
        trimesh.load(
            file_obj= os.path.relpath(stl),
            process=False,
            validate=True
        )
    )

## Load wires and mesh objects into memory

First, load all wires back into a numpy array

In [128]:
contours = []
with open(contour_file) as f:
    wire = f.readline()
    while wire:
        wire = wire.replace('\n','').replace(';',',').split(',')
        raw = [float(xyz) for xyz in wire]
        contours.append(np.reshape(raw,(-1,3)))
        wire = f.readline()
contours = np.array(contours,dtype=object)

Next, find which mesh object corresponds to which wire. Do this by finding the closest distance from each point on the wire to each mesh object, and take the mesh object that's closest. This is an expensive operation, which means it may take a few minutes to run.

To help things run faster, it decimates the contour fed into the proximity.closest_point function

    contour[::4]

In [129]:
contour_address = []
plane_r = np.zeros(len(planes))
for contour in contours:
    for i,plane in enumerate(planes):
        _,r,_ =trimesh.proximity.closest_point(plane,contour[::4])
        plane_r[i]=np.mean(r)
    closest = np.reshape(np.argwhere(plane_r==plane_r.min()),(1,-1))[0]
    contour_address.append(closest[0])
    assert len(closest)==1, "There should only be one closest mesh, not %d" % len(closest)

contour_address = np.array(contour_address,dtype=int)

## Generate edge files for each plane

In [151]:
def write(f,color,scaledpoints,Vm=[],direction=True,unit='cm'):
    x0,y0 = scaledpoints[0,:]
    if Vm:
        f.write(f'\\node[draw] at ({x0}{unit}, {y0}{unit}) {{Vm:1.2f}};\n')
    f.write(f'\\draw[{color}] ({x0:1.2f}{unit}, {y0:1.2f}{unit})\n')
    for x,y in scaledpoints[1:,:]:
        if direction:
            stepsize = np.sqrt((x-x0)**2+(y-y0)**2)
            if stepsize>1: #centimeter
                decor = 'node[sloped,pos=0.5,allow upside down]{\\ArrowIn}'
            else:
                decor = 'node[sloped,pos=0.5,allow upside down]{\\arrowIn}'
        else:
            decor = ''
        x0=x; y0=y;
        f.write(f'--({x:1.2f}{unit},{y:1.2f}{unit}) {decor}\n')


def write_loop(f,color,scaledpoints,Vm=[],direction=True,unit='cm'):
    write(f,color,scaledpoints,Vm,direction,unit)
    f.write('--cycle;\n')

def write_line(f,color,scaledpoints,Vm=[],direction=True,unit='cm'):
    write(f,color,scaledpoints,Vm,direction,unit)
    f.write(';\n')

def write_boundaries(index,coords,scale):
    # write_boundaries(index, coords, scale)
    # 
    # Takes mesh edges from bfieldtools and generates a tikz file to render those edges.
    color = 'color=boundaryBlue'
    bound = os.path.normpath(f'WireWinder/boundary-{index}.tex')
    with open(bound,'a') as f:
        write_line(f,color,coords,direction=False)


# Make a file for each face's outline.

These will be rendered on individual pages.

In [152]:
ij = []
for facecount,plane in enumerate(planes):

    nx = np.unique(plane.vertices[:,0])
    ny = np.unique(plane.vertices[:,1])
    nz = np.unique(plane.vertices[:,2])

    if len(nx)==1:
        i = 1
        j = 2
    elif len(ny)==1:
        i = 0
        j = 2
    elif len(nz)==1:
        i=0
        j=1
    else:
        assert True, "Mesh object is not simple plane. Might need to transform or unwrap more carefully."
    
    ij.append((i,j))
    
    loopz = []
    path=plane.outline()
    for line in path.entities:
        verts = plane.vertices[line.points][:,(i,j)]
        write_boundaries(facecount,verts*100,'cm')

open file for one face, slice to its windings, export them, repeat

In [137]:
color = 'color=wireRed'
for index,xy in enumerate(ij):
    with open(os.path.normpath(f'WireWinder/closed-loops-{index}.tex'),'a') as f:
        for contour in contours[contour_address==index]:
            write_loop(f,color,contour[:,xy]*100)